## Data Collection

weather data is collected from open-Meteo API and the weather dataset for hourly updates (2015-2017) is created for Mumbai City 

### a. Weather Data

In [4]:
import requests
import pandas as pd

# Mumbai coordinates and date range
lat, lon = 19.0760, 72.8777
start_date, end_date = "2015-01-01", "2017-12-31"

# API endpoint: hourly variables
url = (
    "https://archive-api.open-meteo.com/v1/archive?"
    f"latitude={lat}&longitude={lon}"
    f"&start_date={start_date}&end_date={end_date}"
    "&hourly=temperature_2m,relative_humidity_2m,precipitation,wind_speed_10m"
    "&timezone=Asia%2FKolkata"
)

# Fetch data
resp = requests.get(url)
resp.raise_for_status()
data = resp.json()["hourly"]

# Build DataFrame
df = pd.DataFrame({
    "datetime": pd.to_datetime(data["time"]),
    "temp": data["temperature_2m"],
    "humidity": data["relative_humidity_2m"],
    "precipitation": data["precipitation"],
    "windspeed": data["wind_speed_10m"]
})

# Save to CSV
out_path = r"D:\Menternship Project\mumbai_weather_2015_2017_hourly.csv"
df.to_csv(out_path, index=False)
print(f"✅ Hourly weather data saved: {out_path}")


✅ Hourly weather data saved: D:\Menternship Project\mumbai_weather_2015_2017_hourly.csv


### b. Events Data

Events Data is a combination of sports,concerts,holidays and public demonstartions for Mumbai city

* Sports - Kaggle Dataset of IPL Matches for Mumbai
* Concerts - scraped data from bookmyshow.com for Mumbai
* Holidays - Kaggle Dataset of Indian Holidays for Mumbai
* Public Demonstrations - Data from News Sources

## Data Integration Pipeline

* Data Integration Pipeline - merging of Traffic, Weather and Events (combining sports,concerts,holidays and public demonstrations) data into a unified dataset


In [10]:
import pandas as pd

# Load all datasets
traffic = pd.read_csv(r'D:\Menternship Project\traffic_features.csv', parse_dates=['timestamp'])
uber = pd.read_csv(r'D:\Menternship Project\uber_mumbai_2015_2017_synthetic.csv', parse_dates=['pickup_datetime'])
weather = pd.read_csv(r'D:\Menternship Project\mumbai_weather_2015_2017_hourly.csv', parse_dates=['datetime'])
concerts = pd.read_csv(r'D:\Menternship Project\concerts.csv', parse_dates=['event_date'])
holidays = pd.read_csv(r'D:\Menternship Project\holidays.csv', parse_dates=['Date'])
matches = pd.read_csv(r'D:\Menternship Project\matches.csv', parse_dates=['date'])
protests = pd.read_csv(r'D:\Menternship Project\public_demonstrations.csv', parse_dates=['Date'])

# --- Step 1: Rename timestamp to datetime for consistency ---
traffic = traffic.rename(columns={'timestamp': 'datetime'})
uber = uber.rename(columns={'pickup_datetime': 'datetime'})
concerts = concerts.rename(columns={'event_date': 'datetime'})
holidays = holidays.rename(columns={'Date' : 'datetime'})
protests = protests.rename(columns={'Date': 'datetime'})

# Extract date column from datetime for all relevant datasets
traffic['date'] = traffic['datetime'].dt.date
uber['date'] = uber['datetime'].dt.date
concerts['date'] = concerts['datetime'].dt.date
holidays['date'] = holidays['datetime'].dt.date
matches['date'] = matches['date'].dt.date  # Already named 'date'
protests['date'] = protests['datetime'].dt.date

# --- Step 2: Merge traffic datasets ---
traffic_combined = pd.concat([traffic, uber], ignore_index=True)

# Ensure datetime column is uniform
traffic_combined['date'] = traffic_combined['datetime'].dt.date

# --- Step 3: Merge weather on 'datetime' (hourly resolution) ---
merged_df = pd.merge(traffic_combined, weather, on='datetime', how='left')

# --- Step 3: Add event flags based on 'date' ---
for df, event_name in [
    (concerts, 'concert'),
    (holidays, 'holiday'),
    (matches, 'sports_event'),
    (protests, 'protest')
]:
    df[event_name] = 1
    merged_df = pd.merge(
        merged_df,
        df[['date', event_name]].drop_duplicates(),
        on='date',
        how='left'
    )

# Fill missing event indicators with 0
merged_df[['concert', 'holiday', 'sports_event', 'protest']] = \
    merged_df[['concert', 'holiday', 'sports_event', 'protest']].fillna(0).astype(int)

# --- Final Clean-up ---
# Optional: drop helper date column
# merged_df.drop(columns=['date'], inplace=True)

# Save the unified dataset
merged_df.to_csv(r'D:\Menternship Project\mumbai_traffic_weather_events_2015_2017.csv', index=False)

print("Merged dataset created successfully.")
merged_df


Merged dataset created successfully.


,datetime,junction_id,vehicle_count,ID,hour,day_of_week,month,lag_1h,lag_24h,lag_168h,...,weather_condition,traffic_level,temp,humidity,precipitation,windspeed,concert,holiday,sports_event,protest
0,2015-01-11 00:00:00,1.0,15.0,2.015110e+10,0.0,6.0,1.0,NaN,NaN,NaN,...,NaN,NaN,22.1,58.0,0.0,9.2,0,0,0,0
1,2015-01-11 01:00:00,1.0,13.0,2.015110e+10,1.0,6.0,1.0,15.0,NaN,NaN,...,NaN,NaN,21.3,61.0,0.0,7.7,0,0,0,0
2,2015-01-11 02:00:00,1.0,10.0,2.015110e+10,2.0,6.0,1.0,13.0,NaN,NaN,...,NaN,NaN,20.2,65.0,0.0,7.4,0,0,0,0
3,2015-01-11 03:00:00,1.0,7.0,2.015110e+10,3.0,6.0,1.0,10.0,NaN,NaN,...,NaN,NaN,19.2,69.0,0.0,7.2,0,0,0,0
4,2015-01-11 04:00:00,1.0,9.0,2.015110e+10,4.0,6.0,1.0,7.0,NaN,NaN,...,NaN,NaN,18.4,72.0,0.0,8.5,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58115,2015-02-01 19:44:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Cloudy,Low,NaN,NaN,NaN,NaN,0,0,0,0
58116,2015-11-16 14:16:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Clear,Low,NaN,NaN,NaN,NaN,0,0,0,0
58117,2016-05-12 05:03:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Clear,Low,NaN,NaN,NaN,NaN,0,0,1,0
58118,2017-06-28 12:38:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Rainy,Medium,NaN,NaN,NaN,NaN,0,0,0,0


## Data Cleaning and Normalization

* Cleaning of the dataset by removing duplicate records and handling missing values
* Normalize or standardize data to bring different variables to a common scale.

In [13]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# --- Step 1: Load dataset ---
merged_df = pd.read_csv(r'D:\Menternship Project\mumbai_traffic_weather_events_2015_2017.csv', parse_dates=['datetime'],low_memory=False
)

# --- Step 2: Remove duplicate rows ---
merged_df = merged_df.drop_duplicates()
print(f"✅ Removed duplicates. Remaining rows: {len(merged_df)}")

# --- Step 3: Handle missing values ---
# Optional: Print shape before dropping
print(f"Initial shape: {merged_df.shape}")

# Drop rows where more than 50% columns are missing (adjustable)
merged_df.dropna(thresh=int(0.5 * merged_df.shape[1]), inplace=True)
print(f"Remaining rows after dropna(thresh=...): {merged_df.shape[0]}")

# Fill missing numeric columns with mean and categorical with mode
for col in merged_df.columns:
    if merged_df[col].dtype in ['float64', 'int64']:
        if merged_df[col].isnull().sum() > 0:
            merged_df[col] = merged_df[col].fillna(merged_df[col].mean())
    else:
        if merged_df[col].isnull().sum() > 0:
            merged_df[col] = merged_df[col].fillna(merged_df[col].mode()[0])

# --- Step 4: Standardize column names ---
merged_df.columns = [col.lower().strip().replace(' ', '_') for col in merged_df.columns]

# --- Step 5: Normalize numeric features (except event flags) ---
event_columns = ['is_sports_event', 'is_concert', 'is_holiday', 'is_protest']

# Detect junction columns, if any
junction_columns = [col for col in merged_df.columns if col.lower().startswith('junction_')]

# Identify numeric columns
numeric_cols = merged_df.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Remove binary/event columns and one-hot/junction columns from normalization
exclude_cols = event_columns + junction_columns
normalize_cols = [col for col in numeric_cols if col not in exclude_cols]

# Apply MinMaxScaler only if rows exist
if not merged_df.empty:
    scaler = MinMaxScaler()
    merged_df[normalize_cols] = scaler.fit_transform(merged_df[normalize_cols])
else:
    print("⚠️ No rows to normalize after cleaning. Skipping MinMaxScaler.")

# --- Step 6: Save the cleaned dataset ---
output_path = r"D:\Menternship Project\final_merged_dataset_cleaned.csv"
merged_df.to_csv(output_path, index=False)

print(f"\n✅ Dataset cleaned, normalized, and saved as '{output_path}'")
merged_df

✅ Removed duplicates. Remaining rows: 58120
Initial shape: (58120, 31)
Remaining rows after dropna(thresh=...): 58120

✅ Dataset cleaned, normalized, and saved as 'D:\Menternship Project\final_merged_dataset_cleaned.csv'


,datetime,junction_id,vehicle_count,id,hour,day_of_week,month,lag_1h,lag_24h,lag_168h,...,weather_condition,traffic_level,temp,humidity,precipitation,windspeed,concert,holiday,sports_event,protest
0,2015-01-11 00:00:00,1.000000,0.078212,0.000000e+00,0.000000,1.000000,0.000000,0.121732,0.121615,0.121531,...,Clear,Medium,0.322727,0.493976,0.000000,0.218527,0.0,0.0,0.0,0.0
1,2015-01-11 01:00:00,1.000000,0.067039,5.120530e-07,0.043478,1.000000,0.000000,0.078212,0.121615,0.121531,...,Clear,Medium,0.286364,0.530120,0.000000,0.182898,0.0,0.0,0.0,0.0
2,2015-01-11 02:00:00,1.000000,0.050279,1.024106e-06,0.086957,1.000000,0.000000,0.067039,0.121615,0.121531,...,Clear,Medium,0.236364,0.578313,0.000000,0.175772,0.0,0.0,0.0,0.0
3,2015-01-11 03:00:00,1.000000,0.033520,1.536159e-06,0.130435,1.000000,0.000000,0.050279,0.121615,0.121531,...,Clear,Medium,0.190909,0.626506,0.000000,0.171021,0.0,0.0,0.0,0.0
4,2015-01-11 04:00:00,1.000000,0.044693,2.048212e-06,0.173913,1.000000,0.000000,0.033520,0.121615,0.121531,...,Clear,Medium,0.154545,0.662651,0.000000,0.201900,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58115,2015-02-01 19:44:00,2.180549,0.121739,6.247813e-01,0.500000,0.500582,0.466878,0.121732,0.121615,0.121531,...,Cloudy,Low,0.536546,0.674737,0.010133,0.273600,0.0,0.0,0.0,0.0
58116,2015-11-16 14:16:00,2.180549,0.121739,6.247813e-01,0.500000,0.500582,0.466878,0.121732,0.121615,0.121531,...,Clear,Low,0.536546,0.674737,0.010133,0.273600,0.0,0.0,0.0,0.0
58117,2016-05-12 05:03:00,2.180549,0.121739,6.247813e-01,0.500000,0.500582,0.466878,0.121732,0.121615,0.121531,...,Clear,Low,0.536546,0.674737,0.010133,0.273600,0.0,0.0,1.0,0.0
58118,2017-06-28 12:38:00,2.180549,0.121739,6.247813e-01,0.500000,0.500582,0.466878,0.121732,0.121615,0.121531,...,Rainy,Medium,0.536546,0.674737,0.010133,0.273600,0.0,0.0,0.0,0.0


* Checking the percentage of missing values in the final cleaned dataset

In [15]:
# checking the percentage of missing values
round(100*(merged_df.isnull().sum()/len(merged_df.index)), 2)


datetime             0.0
junction_id          0.0
vehicle_count        0.0
id                   0.0
hour                 0.0
day_of_week          0.0
month                0.0
lag_1h               0.0
lag_24h              0.0
lag_168h             0.0
is_weekend           0.0
date                 0.0
trip_id              0.0
dropoff_datetime     0.0
pickup_location      0.0
dropoff_location     0.0
fare_amount          0.0
surge_multiplier     0.0
trip_status          0.0
trip_duration_min    0.0
distance_km          0.0
weather_condition    0.0
traffic_level        0.0
temp                 0.0
humidity             0.0
precipitation        0.0
windspeed            0.0
concert              0.0
holiday              0.0
sports_event         0.0
protest              0.0
dtype: float64